# 加速：多線程爬蟲



* 了解知乎 API 使用方式與回傳內容
* 撰寫程式存取 API 且添加標頭

## 作業目標

* 找一個之前實作過的爬蟲改用多線程改寫，比較前後時間的差異。





In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import _thread

In [2]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
#    for p in news_content:
#        if ((p.string) is not None):
#            print(p.string)

In [3]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

In [4]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

In [5]:
#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("return document.body.scrollHeight;")

while True:
    # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page.
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        print("E")
        break

    last_height = new_height
    print(".", end="")

............E


In [6]:
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

In [7]:
# 關閉瀏覽器
browser.quit();

### Singal thread

In [8]:
startTime = time.time()

for i, news_item in enumerate(news_block):
    news_body = news_item.find('h3')
    #print("----------------------------------------------------------------------")
    #print("\n[%d] %s\n" % (i, news_body.a.string))
    
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)
    finishTime = time.time()

print(finishTime - startTime)

13.189558744430542


### Multi thread

In [9]:
startTime = time.time()

for i, news_item in enumerate(news_block):
    news_body = news_item.find('h3')
    #print("----------------------------------------------------------------------")
    #print("\n[%d] %s\n" % (i, news_body.a.string))
    
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    #getNewsDetailContent(externalLink)
    _thread.start_new_thread(getNewsDetailContent,(externalLink, ))
    finishTime = time.time()

print(finishTime - startTime)

0.04527473449707031
